https://adventofcode.com/2018/day/7

Objective: define a class to represent a step. Each instance of a step will store instance variables to keep track of what other steps it depends on, and what other steps depend on it.

dictionary, key:value , step is the key and instance is the value

In [1]:
import re

In [2]:
ord('A')

65

In [3]:
class Step():
    def __init__(self, name):
        self.name = name
        self.duration = ord(name) - ord('A') + 61
        self.still_waiting_on = set()
        self.waiting_on_me = set()
        
    def __repr__(self):
        return 'My name is {0}.\nI\'m out here tryna function.\nI\'m waiting on {1}.\n{2} are waiting on me'.format(self.name, self.still_waiting_on, self.waiting_on_me)
        


In [4]:
with open('input_D7.txt') as file:
    all_steps = set()
    for line in file:
        for match in re.finditer('[Ss]tep (\w)', line):
            all_steps.add(match.group(1))
            
#all_steps is just the alphabet

In [5]:
step_dictionary = {name: Step(name) for name in all_steps}

with open('input_D7.txt') as file:
    for line in file:
        uppercase_match = re.search('Step (\w)', line).group(1)
        lowercase_match = re.search('step (\w)', line).group(1)
        
        step_dictionary[uppercase_match].waiting_on_me.add(lowercase_match)
        step_dictionary[lowercase_match].still_waiting_on.add(uppercase_match)
        

correct_order = []

while len(step_dictionary) > 0:
    ready_to_go = set(name for name in step_dictionary if len(step_dictionary[name].still_waiting_on) == 0)
    current_name = min(ready_to_go)
    correct_order.append(current_name)

    current_step = step_dictionary.pop(current_name)


    for letter in current_step.waiting_on_me:
        step_dictionary[letter].still_waiting_on.remove(current_step.name)

print(''.join(correct_order))


AEMNPOJWISZCDFUKBXQTHVLGRY


when executing a step, add it to the ordered list, then pull out that steps "waiting on me" set, then for each letter in that set, go into that letter's still waiting on set and remove the step. lastly, remove the step from the dictionary

Part two:

Want to add to the step instances a variable called duration that stores how long each one takes

write a function called execute_task that takes a letter and pops it out of our dictionary, and removes it from the other things waiting on it.

separately, called get_next_available which returns a step object that is next available.

In [7]:
from queue import PriorityQueue

In [8]:
def execute_task(name):
    current_step = step_dictionary.pop(name)
    for letter in current_step.waiting_on_me:
        step_dictionary[letter].still_waiting_on.remove(current_step.name)

In [9]:
def next_available():
    ready_to_go = set(name for name in step_dictionary if len(step_dictionary[name].still_waiting_on) == 0)
    return min(ready_to_go)

In [10]:
elf_1 = []
elf_2 = []
elf_3 = []
elf_4 = []

three things to keep track one:
 - Done in first part, which tasks need others to be compelted (dictionary of Steps)
 - How many elves we have, and how many are busy (set of available elves)
 - The specific times tasks finish in given when they start (priorityqueue)

make a new class called Step_scheduler. define different methods to tell us things like what Steps are available, or what the next event to come off priority queue